In [ ]:
import torch
import torchvision
from torchvision import transforms
import torchmetrics
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger
from tqdm.notebook import tqdm
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def load_file(path):
    return np.load(path).astype(np.float32)

In [ ]:
train_transforms = transforms.Compose([
                                    transforms.ToTensor(),  
                                    transforms.Normalize(0.49, 0.248),  
                                    transforms.RandomAffine( 
                                        degrees=(-5, 5), translate=(0, 0.05), scale=(0.9, 1.1)),
                                        transforms.RandomResizedCrop((224, 224), scale=(0.35, 1))

])

val_transforms = transforms.Compose([
                                    transforms.ToTensor(),  
                                    transforms.Normalize([0.49], [0.248]),  
])





In [ ]:
train_dataset=torchvision.datasets.DatasetFolder(
    "Processed/train/",
    loader=load_file,extensions=".npy",transform=train_transforms)
val_dataset=torchvision.datasets.DatasetFolder(
    "Processed/val/",
    loader=load_file,extensions=".npy",transform=val_transforms)



In [ ]:
fig,axis=plt.subplots(2,2,figsize=(9,9))
for i in range(2):
    for j in range(2):
        random_index=np.random.randint(0,20000)
        x_ray,label=train_dataset[random_index]
        axis[i][j].imshow(x_ray[0],cmap="bone")
        axis[i][j].set_title(f"Label:{label}")

In [ ]:
print(f"train_dataset has {len(train_dataset)} images")
print(f"val_dataset has {len(val_dataset)} images")

In [ ]:
train_loader=torch.utils.data.DataLoader(train_dataset,batch_size=64,num_workers=0,shuffle=True)
val_loader=torch.utils.data.DataLoader(val_dataset,batch_size=64,num_workers=0,shuffle=False)

In [ ]:
torchvision.models.resnet18()


In [ ]:
np.unique(train_dataset.targets,return_counts=True)

In [ ]:
from torchmetrics.classification import Accuracy,AUROC
class PneumoniaModel(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.model=torchvision.models.resnet18()
        self.model.conv1=torch.nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        self.model.fc=torch.nn.Linear(in_features=512, out_features=1, bias=True)


        self.optimizer=torch.optim.Adam(self.model.parameters(),lr=1e-4)
        self.loss_fn=torch.nn.BCEWithLogitsLoss(pos_weight=torch.tensor([3]))

        self.train_acc=Accuracy(task="binary")
        self.val_acc=Accuracy(task="binary")
        self.val_auc =AUROC(task="binary")

    def forward(self, data):
        pred=self.model(data)
        return pred
    

    def training_step(self,batch,batch_idx):
        x_ray,label=batch
        label=label.float()
        pred=self(x_ray).squeeze(1)
        loss=self.loss_fn(pred,label)

        probs=torch.sigmoid(pred)
        acc=self.train_acc(probs,label.int())

        self.log("Train Loss",loss,prog_bar=True,on_step=False,on_epoch=True)
        self.log("Step Train ACC",acc,prog_bar=True,on_step=False,on_epoch=True)
        
        return loss
    
    
    def on_training_epoch_end(self):
        self.log("Train ACC",self.train_acc.compute())
        

    def validation_step(self,batch,batch_idx):
        x_ray,label=batch
        label=label.float()
        pred=self(x_ray).squeeze(1)        
        loss=self.loss_fn(pred,label)

        probs=torch.sigmoid(pred)

        acc=self.val_acc(probs,label.int())
        self.log("Val Loss",loss,prog_bar=True,on_step=False, on_epoch=True)
        self.log("val_acc",  acc,   prog_bar=True, on_step=False, on_epoch=True)
        self.val_auc.update(probs, label.int())

        
        

    def on_validation_epoch_end(self):
        auroc = self.val_auc.compute()
        self.log("val_auroc", auroc, prog_bar=True)
        self.val_auc.reset()



    def configure_optimizers(self):
        return [self.optimizer]


In [ ]:
model=PneumoniaModel()

In [ ]:
from pytorch_lightning.callbacks import ModelCheckpoint,EarlyStopping
checkpoint_callback_acc=ModelCheckpoint(
    monitor="val_acc",
    save_top_k=1,
    mode="max",
    filename="best-acc-{epoch:02d}-{val_acc:.4f}-{val_auroc:.4f}"
)
early_stop = EarlyStopping(monitor="val_auroc", mode="max", patience=5)
checkpoint_callback_auroc=ModelCheckpoint(
    monitor="val_auroc",
    save_top_k=1,
    mode="max",
    filename="best-auroc-{epoch:02d}-{val_acc:.4f}-{val_auroc:.4f}"
)

In [ ]:

trainer=pl.Trainer(accelerator="gpu",devices=1,
        logger=TensorBoardLogger(save_dir="./logs"),log_every_n_steps=1,max_epochs=35,
        callbacks=[checkpoint_callback_acc,checkpoint_callback_auroc,early_stop])

       
        

In [ ]:
import numpy as np

train_labels = [y for _, y in train_dataset]
val_labels   = [y for _, y in val_dataset]

print("Train positives:", np.sum(train_labels), 
      "negatives:", len(train_labels) - np.sum(train_labels))
print("Val positives:", np.sum(val_labels), 
      "negatives:", len(val_labels) - np.sum(val_labels))


In [ ]:
print(f"Val_loader has {len(val_loader)} images")
print(f"train_loader has {len(train_loader)} images")

In [ ]:
trainer.fit(model,train_loader,val_loader)

In [ ]:
print("Best ACC ckpt:", checkpoint_callback_acc.best_model_path)
print("Best AUROC ckpt:", checkpoint_callback_auroc.best_model_path)

In [ ]:
import os
print(os.getcwd())


In [ ]:
import glob
print(glob.glob("**/*.ckpt", recursive=True))


In [ ]:
import os

ckpt_path = r"C:\Users\milos\Projects\rsna-pneumonia-detection-challenge\logs\lightning_logs\version_5\checkpoints\best-auroc-epoch=25-val_acc=0.8171-val_auroc=0.8936.ckpt"

print("Exists?", os.path.exists(ckpt_path))
print("Dir listing:", os.listdir(os.path.dirname(ckpt_path)))


In [ ]:
import os
device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
ckpt_path = r"C:\Users\milos\Projects\rsna-pneumonia-detection-challenge\logs\lightning_logs\version_5\checkpoints\best-auroc-epoch=25-val_acc=0.8171-val_auroc=0.8936.ckpt"


model=PneumoniaModel.load_from_checkpoint(ckpt_path)
model.eval()
model.to(device)

In [ ]:
preds=[]
labels=[]

with torch.no_grad():
    for data,label in tqdm(val_dataset):
        data=data.to(device).float().unsqueeze(0)
        pred=torch.sigmoid(model(data)[0].cpu())
        preds.append(pred)
        labels.append(label)

preds=torch.tensor(preds)
labels=torch.tensor(labels).int()

In [ ]:
import torch
import torchmetrics
from torchmetrics.classification import Accuracy, Precision, Recall, ConfusionMatrix
acc=Accuracy(task="binary")(preds,labels)
precision=Precision(task="binary")(preds,labels)
recall=Recall(task="binary")(preds,labels)
cm=ConfusionMatrix(task="binary", num_classes=2)(preds,labels)
auroc=AUROC(task="binary")(preds,labels)

print(f"Val Accuracy:  {acc:.4f}")
print(f"Val Precision: {precision:.4f}")
print(f"Val Recall:    {recall:.4f}")
print("Confusion Matrix:")
print(cm)
print(f"AUROC :  {auroc:.4f}")